<a href="https://colab.research.google.com/github/Swarnlataaa/Automation_py/blob/main/assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:


!pip install kagglehub ultralytics tqdm opencv-python matplotlib


In [11]:

import kagglehub
import os
import shutil
import xml.etree.ElementTree as ET
import cv2
from tqdm import tqdm

dataset_path = kagglehub.dataset_download("dataclusterlabs/palm-and-gloves-dataset")
print("Dataset downloaded to:", dataset_path)


Using Colab cache for faster access to the 'palm-and-gloves-dataset' dataset.
Dataset downloaded to: /kaggle/input/palm-and-gloves-dataset


In [12]:


root = os.path.join(dataset_path, "palm_100")
files = os.listdir(root)

print("Total files:", len(files))
print("Example:", files[:10])


Total files: 200
Example: ['1591361224424.xml', '1591587457483.jpg', '1590231897804.jpg', '1591340738722.xml', '1590231921683.xml', '1591588350659.jpg', '1591580532547.xml', '1591588302665.xml', '1591587224729.jpg', '1591580065561.jpg']


In [13]:


# CLASS MAPPING (IMPORTANT)
CLASS_MAP = {
    "glove": 0,
    "human_palm": 1      # will be bare_hand
}

# Create YOLO dataset structure
if os.path.exists("yolo_dataset"):
    shutil.rmtree("yolo_dataset")

os.makedirs("yolo_dataset/images/train", exist_ok=True)
os.makedirs("yolo_dataset/images/val", exist_ok=True)
os.makedirs("yolo_dataset/labels/train", exist_ok=True)
os.makedirs("yolo_dataset/labels/val", exist_ok=True)

images = [f for f in files if f.endswith(".jpg")]
split_index = int(0.8 * len(images))

def voc_to_yolo(xml_path, img_shape):
    tree = ET.parse(xml_path)
    root_xml = tree.getroot()

    h, w = img_shape[:2]
    labels = []

    for obj in root_xml.findall("object"):

        cls = obj.find("name").text.strip().lower()

        if cls not in CLASS_MAP:
            continue

        cls_id = CLASS_MAP[cls]

        bbox = obj.find("bndbox")
        xmin = float(bbox.find("xmin").text)
        ymin = float(bbox.find("ymin").text)
        xmax = float(bbox.find("xmax").text)
        ymax = float(bbox.find("ymax").text)

        x_center = ((xmin + xmax) / 2) / w
        y_center = ((ymin + ymax) / 2) / h
        bw = (xmax - xmin) / w
        bh = (ymax - ymin) / h

        labels.append(f"{cls_id} {x_center} {y_center} {bw} {bh}")

    return labels


for idx, img in enumerate(tqdm(images)):

    img_path = os.path.join(root, img)
    xml_path = os.path.join(root, img.replace(".jpg", ".xml"))

    if not os.path.exists(xml_path):
        continue

    img_data = cv2.imread(img_path)
    labels = voc_to_yolo(xml_path, img_data.shape)

    if idx < split_index:
        img_out = f"yolo_dataset/images/train/{img}"
        lbl_out = f"yolo_dataset/labels/train/{img.replace('.jpg','.txt')}"
    else:
        img_out = f"yolo_dataset/images/val/{img}"
        lbl_out = f"yolo_dataset/labels/val/{img.replace('.jpg','.txt')}"

    shutil.copy2(img_path, img_out)

    with open(lbl_out, "w") as f:
        for l in labels:
            f.write(l + "\n")

print("XML → YOLO conversion DONE!")


100%|██████████| 100/100 [00:10<00:00,  9.20it/s]

XML → YOLO conversion DONE!


In [14]:


yaml_content = """
path: yolo_dataset
train: images/train
val: images/val

names:
  0: gloved_hand
  1: bare_hand
"""

with open("dataset.yaml", "w") as f:
    f.write(yaml_content)

print("dataset.yaml created")


dataset.yaml created


In [15]:

from ultralytics import YOLO

model = YOLO("yolov8s.pt")

results = model.train(
    data="dataset.yaml",
    epochs=15,
    imgsz=640,
)

print("Training finished!")


Ultralytics 8.3.240 🚀 Python-3.12.12 torch-2.9.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=15, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained

In [16]:


import os

os.makedirs("test_images", exist_ok=True)
os.makedirs("output", exist_ok=True)
os.makedirs("logs", exist_ok=True)

print("Upload .jpg images into test_images/ folder to run inference")


Upload .jpg images into test_images/ folder to run inference


In [17]:

import json

def detect(conf=0.4):
    model = YOLO("runs/detect/train/weights/best.pt")

    for img_file in os.listdir("test_images"):

        if not img_file.endswith(".jpg"):
            continue

        img_path = f"test_images/{img_file}"
        results = model.predict(img_path, conf=conf)

        # Save annotated image
        annotated = results[0].plot()
        cv2.imwrite(f"output/{img_file}", annotated)

        # Create JSON log
        log = {
            "filename": img_file,
            "detections": []
        }

        for box in results[0].boxes:
            cls = int(box.cls)
            conf_score = float(box.conf)
            bbox = box.xyxy.tolist()[0]

            log["detections"].append({
                "label": model.names[cls],
                "confidence": round(conf_score, 3),
                "bbox": bbox
            })

        with open(f"logs/{img_file.replace('.jpg','.json')}", "w") as f:
            json.dump(log, f, indent=2)

    print("Inference done!")


In [18]:
detect()


Inference done!


In [19]:
!ls test_images


In [20]:
import os, shutil, random

src = "yolo_dataset/images/val"
for img in random.sample(os.listdir(src), 5):
    shutil.copy(f"{src}/{img}", "test_images/")


In [21]:
detect()
!ls output



image 1/1 /content/test_images/1591589300034.jpg: 640x480 1 bare_hand, 47.2ms
Speed: 2.9ms preprocess, 47.2ms inference, 1.5ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/test_images/1591588006036.jpg: 640x480 1 bare_hand, 12.2ms
Speed: 2.5ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/test_images/1589894383837.jpg: 640x480 2 bare_hands, 12.3ms
Speed: 4.1ms preprocess, 12.3ms inference, 2.2ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/test_images/1591580426512.jpg: 640x480 1 bare_hand, 12.2ms
Speed: 2.7ms preprocess, 12.2ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 480)

image 1/1 /content/test_images/1591542369428.jpg: 640x480 1 bare_hand, 12.1ms
Speed: 2.9ms preprocess, 12.1ms inference, 1.4ms postprocess per image at shape (1, 3, 640, 480)
Inference done!
1589894383837.jpg  1591580426512.jpg  1591589300034.jpg
1591542369428.jpg  1591588006036.jpg


In [ ]:
#if you want to automatically test on validation images
def detect(conf=0.4):
    model = YOLO("runs/detect/train/weights/best.pt")
    img_folder = "yolo_dataset/images/val"


In [22]:
!zip

Copyright (c) 1990-2008 Info-ZIP - Type 'zip "-L"' for software license.
Zip 3.0 (July 5th 2008). Usage:
zip [-options] [-b path] [-t mmddyyyy] [-n suffixes] [zipfile list] [-xi list]
  The default action is to add or replace zipfile entries from list, which
  can include the special name - to compress standard input.
  If zipfile and list are omitted, zip compresses stdin to stdout.
  -f   freshen: only changed files  -u   update: only changed or new files
  -d   delete entries in zipfile    -m   move into zipfile (delete OS files)
  -r   recurse into directories     -j   junk (don't record) directory names
  -0   store only                   -l   convert LF to CR LF (-ll CR LF to LF)
  -1   compress faster              -9   compress better
  -q   quiet operation              -v   verbose operation/print version info
  -c   add one-line comments        -z   add zipfile comment
  -@   read names from stdin        -o   make zipfile as old as latest entry
  -x   exclude the following nam

In [23]:
!zip -r glove_results.zip output logs


  adding: output/ (stored 0%)
  adding: output/1591589300034.jpg (deflated 2%)
  adding: output/1591588006036.jpg (deflated 0%)
  adding: output/1589894383837.jpg (deflated 4%)
  adding: output/1591580426512.jpg (deflated 3%)
  adding: output/1591542369428.jpg (deflated 3%)
  adding: logs/ (stored 0%)
  adding: logs/1591588006036.json (deflated 35%)
  adding: logs/1591589300034.json (deflated 36%)
  adding: logs/1591542369428.json (deflated 35%)
  adding: logs/1591580426512.json (deflated 36%)
  adding: logs/1589894383837.json (deflated 52%)


In [24]:
from google.colab import files
files.download("glove_results.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>